# Hyperparameter Tuning using Keras Tuner and Tensorflow

# Introduction
KerasTuner is a general-purpose hyperparameter tuning library. It has strong integration with Keras workflows, but it isn't limited to them: you could use it to tune scikit-learn models, or anything else. In this lab, you will see how to tune model architecture, training process, and data preprocessing steps with KerasTuner.

There are some advanced hyperparameter tuning algorithms, including Random serach tuner, Bayesian hyperparameter optimization, Hyperband, Sklearn tuner. All of these are implemented inside the [keras tuner package](https://keras.io/keras_tuner/).

### Advantages of Keras Tuner


1.   Ease of use

2. Integrates into your existing deep learning training pipeline with minimal code changes
3. Implements novel hyperparameter tuning algorithms
4. Can boost accuracy with minimal effort on your part




In [ ]:
!pip install keras-tuner -q

     |████████████████████████████████| 135 kB 14.0 MB/s 
     |████████████████████████████████| 1.6 MB 69.9 MB/s 


In [ ]:
from tensorflow import keras
from tensorflow.keras import layers


def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Flatten())
    model.add(
        layers.Dense(
            # Define the hyperparameter.
            units=hp.Int("units", min_value=32, max_value=512, step=32),
            activation="relu",
        )
    )
    model.add(layers.Dense(10, activation="softmax"))
    model.compile(
        optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"],
    )
    return model

In [ ]:
#quickly test if the model builds successfully.
import keras_tuner

build_model(keras_tuner.HyperParameters())

In [ ]:
tuner = keras_tuner.RandomSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=3,  #The total number of trials to run during the search.
    executions_per_trial=2, #The number of models that should be built and fit for each trial.
    overwrite=True, #Control whether to overwrite the previous results, overwrite=True to start a new search and ignore any previous results.
    directory="my_dir",#A path to a directory for storing the search results.
    project_name="helloworld", #The name of the sub-directory in the directory.
)

In [ ]:
tuner.search_space_summary() #print a summary of the search space

Search space summary
Default search space size: 1
units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}


In [ ]:
#Before starting the search, let's prepare the MNIST dataset.
from tensorflow import keras
import numpy as np

(x, y), (x_test, y_test) = keras.datasets.mnist.load_data()

x_train = x[:-10000]
x_val = x[-10000:]
y_train = y[:-10000]
y_val = y[-10000:]

x_train = np.expand_dims(x_train, -1).astype("float32") / 255.0
x_val = np.expand_dims(x_val, -1).astype("float32") / 255.0
x_test = np.expand_dims(x_test, -1).astype("float32") / 255.0

num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

11501568/11490434 [==============================] - 0s 0us/step


Then, start the search for the best hyperparameter configuration. All the arguments passed to search is passed to model.fit() in each execution. Remember to pass validation_data to evaluate the model.

In [ ]:
tuner.search(x_train, y_train, epochs=2, validation_data=(x_val, y_val))

Trial 3 Complete [00h 00m 42s]
val_accuracy: 0.9748000204563141

Best val_accuracy So Far: 0.9748000204563141
Total elapsed time: 00h 01m 24s


In [ ]:
tuner.results_summary() # print a summary of the search results.

Results summary
Results in my_dir/helloworld
Showing 10 best trials
Trial summary
Hyperparameters:
units: 480
Score: 0.9748000204563141
Trial summary
Hyperparameters:
units: 224
Score: 0.9711999893188477
Trial summary
Hyperparameters:
units: 32
Score: 0.9532999992370605


## Query the results
When search is over, you can retrieve the best model(s). The model is saved at its best performing epoch evaluated on the validation_data.

In [ ]:
# Get the top 2 models.
models = tuner.get_best_models(num_models=2)
best_model = models[0]
# Build the model.
# Needed for `Sequential` without specified `input_shape`.
best_model.build(input_shape=(None, 28, 28))
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 480)               376800    
                                                                 
 dense_1 (Dense)             (None, 10)                4810      
                                                                 
Total params: 381,610
Trainable params: 381,610
Non-trainable params: 0
_________________________________________________________________


## Retrain the model
If you want to train the model with the entire dataset, you may retrieve the best hyperparameters and retrain the model by yourself.

In [ ]:
# Get the top 2 hyperparameters.
best_hps = tuner.get_best_hyperparameters(5)
# Build the model with the best hp.
model = build_model(best_hps[0])
# Fit with the entire dataset.
x_all = np.concatenate((x_train, x_val))
y_all = np.concatenate((y_train, y_val))
model.fit(x=x_all, y=y_all, epochs=1)

1875/1875 [==============================] - 8s 4ms/step - loss: 0.2042 - accuracy: 0.9399


In [ ]:
# <put your resolve here >
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Flatten())
    layer_num = hp.Int("layer", min_value=1, max_value=9, step=1)
    for _ in range(layer_num):
      model.add(
          layers.Dense(
              units=hp.Int("units", min_value=32, max_value=512, step=32),
              activation=hp.Choice("activation", values = ["relu", "tanh"]),
          )
      )

    model.add(keras.layers.Dropout(0.15))
    model.add(layers.Dense(10, activation="softmax"))

    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate), loss="categorical_crossentropy", metrics=["accuracy"],
    )
    return model

In [ ]:
# <put your resolve here >
tuner = keras_tuner.Hyperband(build_model,objective='val_accuracy',max_epochs=5,factor=3,directory="/content/keras_tuner_test",project_name= "kt_hyperband")

In [ ]:
# <put your resolve here >
from sklearn.model_selection import train_test_split

(x, y), (x_test, y_test) = keras.datasets.mnist.load_data()

x_train, X_rem, y_train, y_rem = train_test_split(x,y, train_size=0.8)

# Now since we want the valid and test size to be equal (10% each of overall data). 
# we have to define valid_size=0.5 (that is 50% of remaining data)
test_size = 0.5
x_val, x_test, y_val, y_test = train_test_split(X_rem,y_rem, test_size=0.5)

x_train = np.expand_dims(x_train, -1).astype("float32") / 255.0
x_val = np.expand_dims(x_val, -1).astype("float32") / 255.0
x_test = np.expand_dims(x_test, -1).astype("float32") / 255.0

num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


In [ ]:
# <put your resolve here >
tuner.search(x_train, y_train, epochs=2, validation_data=(x_val, y_val))

Trial 10 Complete [00h 02m 23s]
val_accuracy: 0.95333331823349

Best val_accuracy So Far: 0.9703333377838135
Total elapsed time: 00h 10m 44s


In [15]:
tuner.results_summary()

Results summary
Results in /content/keras_tuner_test/kt_hyperband
Showing 10 best trials
Trial summary
Hyperparameters:
layer: 2
units: 320
activation: tanh
learning_rate: 0.001
tuner/epochs: 5
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.9703333377838135
Trial summary
Hyperparameters:
layer: 6
units: 480
activation: relu
learning_rate: 0.001
tuner/epochs: 5
tuner/initial_epoch: 2
tuner/bracket: 1
tuner/round: 1
tuner/trial_id: 0001
Score: 0.9671666622161865
Trial summary
Hyperparameters:
layer: 6
units: 480
activation: relu
learning_rate: 0.001
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 1
tuner/round: 0
Score: 0.9614999890327454
Trial summary
Hyperparameters:
layer: 7
units: 480
activation: tanh
learning_rate: 0.0001
tuner/epochs: 5
tuner/initial_epoch: 2
tuner/bracket: 1
tuner/round: 1
tuner/trial_id: 0004
Score: 0.9580000042915344
Trial summary
Hyperparameters:
layer: 7
units: 480
activation: tanh
learning_rate: 0.001
tuner/epochs: 5
tuner/initial_epoch

In [16]:
# Get the top 2 models.
models = tuner.get_best_models(num_models=2)
best_model = models[0]
# Build the model.
# Needed for `Sequential` without specified `input_shape`.
best_model.build(input_shape=(None, 28, 28))
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 320)               251200    
                                                                 
 dense_1 (Dense)             (None, 320)               102720    
                                                                 
 dropout (Dropout)           (None, 320)               0         
                                                                 
 dense_2 (Dense)             (None, 10)                3210      
                                                                 
Total params: 357,130
Trainable params: 357,130
Non-trainable params: 0
_________________________________________________________________
